In [3]:
!pip install --upgrade pip
!pip install gradio
!pip install --upgrade transformers sentencepiece datasets[audio]
!pip install transformers torchaudio
!pip install -q kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [4]:
import gradio as gr
import torch
from transformers import pipeline, AutoProcessor, WhisperForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM
import torchaudio
from TTS.api import TTS
from datasets import load_dataset

In [ ]:
'''
# Colab login
from google.colab import userdata
from huggingface_hub import login

login(userdata.get('huggingface'))
'''

In [ ]:
# Kaggle login⁠
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("NLP project") ⁠

In [6]:
# === 1. Load STT model (Whisper) ===
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
whisper_processor = AutoProcessor.from_pretrained("openai/whisper-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [7]:
pipe = pipeline(
    "text-generation",
    model='meta-llama/Llama-3.2-1B-Instruct',
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
import torch
import soundfile as sf
from datasets import load_dataset
from kokoro import KPipeline
from IPython.display import Audio, display

pipeline = KPipeline(lang_code='a')

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [21]:
def from_text_to_speech_with_embedding(text, embedding):
    """Convert text into audio using Kokoro + speaker embedding."""
    audio_path = '/tmp/generated_audio.wav'

    generator = pipeline(text, voice='af_heart')

    for i, (generated_sentence, phonemes, audio) in enumerate(generator):
        print(f"▶️ [{i}] Generato: {generated_sentence}")
        sf.write(audio_path, audio, 24000)

    return audio_path

In [22]:
def transcribe(audio):
    speech, sr = torchaudio.load(audio)
    if sr != 16000:
        speech = torchaudio.functional.resample(speech, orig_freq=sr, new_freq=16000)
    input_features = whisper_processor(speech.squeeze(0), sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = whisper_model.generate(input_features)
    transcription = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription

def chat(user_input):
    messages = [{"role": "user", "content": user_input}]
    outputs = pipe(messages, max_new_tokens=256)
    return outputs[0]["generated_text"][-1]["content"]

def generate_response(text=None, audio=None):
    if audio:
        text = transcribe(audio)

    print(f"User: {text}")
    response = chat(text)

    audio_path = from_text_to_speech_with_embedding(response, speaker_embedding)

    return response, audio_path


In [23]:
with gr.Blocks() as demo:
    gr.Markdown("## 🎙️🧠 Speech+Text Chatbot con Modelli Open Source")

    with gr.Row():
        text_input = gr.Textbox(label="Write a message")
        audio_input = gr.Audio(type="filepath", label="...or talk")

    submit_btn = gr.Button("Send")

    with gr.Row():
        text_output = gr.Textbox(label="Bot answer")
        audio_output = gr.Audio(label="Listen the answer", type="filepath")

    submit_btn.click(
        fn=generate_response,
        inputs=[text_input, audio_input],
        outputs=[text_output, audio_output]
    )

    # Input reset
    submit_btn.click(
        fn=lambda: ("", None),
        inputs=None,
        outputs=[text_input, audio_input],
        queue=False
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e3725a5ff62c1ad36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
